<a href="https://colab.research.google.com/github/dina-lab3D/ContactNet/blob/master/ContactNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#Antigen-antibody docking tutorial
<img src="https://drive.google.com/uc?export=view&id=1J6FA12dZKcf7ydSQdnFo8b7bwi-NehB5" width="300">
<img src="https://drive.google.com/uc?export=view&id=1J83QaZWoK1Xi2-dqt7o8jdImcJ8HDJf-" width="150">

In this tutorial we use a docking software to dock a modeled antibody to an antigen. We will work in 2 steps:


1. Docking using PatchDock
2. Scoring using interface scoring function ContactNet



In [ ]:
#@title Download docking software - PatchDock
!wget https://github.com/dina-lab3D/tutorials/raw/main/PatchDock/pd.zip
!unzip pd.zip

In [ ]:
#@title Download and install ContactNet
%cd /content/
!git clone https://github.com/dina-lab3D/ContactNet --quiet
!pip install tensorflow_addons
!pip install wandb
%cd ContactNet/src/ComplexDistogram/
!cmake .
!make


In [ ]:
#@title Prepare input for docking in the example folder
%cd /content/
%mkdir example
!cp /content/ContactNet/example/*.pdb /content/example/

/content


In [ ]:
#@title run PatchDock
%cd /content/example/
# dock the antibody model to the antigen, final clustering of 4A, "AA" tells PatchDock to use antigen-antibody docking and focus on nanobody CDRs
! ../PatchDock/buildParams.pl 1kip_Ag.pdb 1kip_AF2_Ab.pdb 4.0 AA
! ../PatchDock/patch_dock.Linux params.txt docking.res
# generate top 10 by geometric score
! ../PatchDock/transOutput.pl docking.res 1 10

In [ ]:
#@title run ContactNet scoring
!python3 /content/ContactNet/data_maker.py 1kip_Ag.pdb 1kip_AF2_Ab.pdb docking.res --trans_num 1000
!python3 /content/ContactNet/predict_prot.py 1kip_Ag.pdb 1kip_AF2_Ab.pdb --trans_num 1000

In [ ]:
#@title plot scores vs. RMSDs (funnel)

# create a file with interface rmsds and the scores
!grep "|" docking.res | awk '{print $1" "$4} ' | tr '(' ' ' | tr ')' ' ' > rmsds
!sed -i 's/|/rmsd/' rmsds
!cut -f2 evaluation > cnet
!paste -d ' ' rmsds cnet | head -1001 > all_scores.txt

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from plotnine import *
from scipy import stats
from scipy.interpolate import make_interp_spline

df = pd.read_csv("all_scores.txt", delimiter=r"\s+")
#print (df)
sns.set_theme(style="whitegrid")
sns.set(font_scale = 2)
sns.set_style("whitegrid")

fig1 = plt.figure(1, figsize=(6, 4), dpi=100)

sns.scatterplot(data = df,x="rmsd", y="score")
plt.show()
plt.savefig('scatter.png')

# reverse ContactNet scores (now lower is better)
#df["score"] = -df["score"]

# select best model
best_model = int(df["#"].iloc[df["score"].argmax()])
print(best_model)




In [ ]:
!pip install py3Dmol
#@title Display 3D structure {run: "auto"}
import py3Dmol
import glob

#@title Display best scoring model
! /content/PatchDock/transOutput.pl docking.res $best_model $best_model
with open("1kip_AF2_Ab.pdb") as ifile:
    true = "".join([x for x in ifile])
with open(f"docking.res.{best_model}.pdb") as ifile:
    predicted = "".join([x for x in ifile])


r,g,b = 255,0,0
print(f"\033[38;2;{r};{g};{b}mPredicted model \033[38;2;255;255;255m")
r,g,b = 0,0,255
print(f"\033[38;2;{r};{g};{b}mNative structure \033[38;2;255;255;255m")
view = py3Dmol.view(width=500, height=500)
view.addModelsAsFrames(predicted)
view.setStyle({'model': 0, "chain": "C"}, {"cartoon": {'arrows':True, 'color': 'gray'}})
view.setStyle({'model': 0, "chain": "A"}, {"cartoon": {'arrows':True, 'color': 'red'}})
view.setStyle({'model': 0, "chain": "B"}, {"cartoon": {'arrows':True, 'color': 'red'}})
view.addModelsAsFrames(true)
view.setStyle({'model': 1}, {"cartoon": {'arrows':True, 'color': 'blue'}})
view.zoomTo()
view.show()
